In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Huntingtons_Disease/GSE147003'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"DNA methylation profiles from a transgenic sheep model of Huntington's disease"
!Series_summary	"An Infinium microarray platform (HorvathMammalMethylChip40) was used to generate DNA methylation data from n=168 blood samples of a transgenic sheep model of Huntington's disease."
!Series_summary	"84 transgenic sheep and age matched control sheep."
!Series_overall_design	"Two genotypes (transgenic) and control.  Roughly sex balanced (half females)."
!Series_overall_design	"Each sample characteristics value represents:"
!Series_overall_design	"characteristics: Basename = usual identifier of the methylation array comprised of the SENTRIX ID (i.e. chip identifier) and position (row and column)."
!Series_overall_design	"characteristics: Age (Years) = Age at the time of blood draw in units of years."
!Series_overall_design	"characteristics: Sex = female denotes female sheep."
!Series_overall_design	"characteristics: Genotype = Transgenic denotes Huntington

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = False  # Dataset is a DNA methylation dataset, not suitable.

# Variable availability
try:
    trait_row = 0 if "genotype huntington" in " ".join([str(v) for v in sample_characteristics_dict[0]]) else None
except KeyError:
    trait_row = None
   
try:
    age_row = 1 if "age (years)" in " ".join([str(v) for v in sample_characteristics_dict[1]]) else None
except KeyError:
    age_row = None

try:
    gender_row = 2 if "Sex" in " ".join([str(v) for v in sample_characteristics_dict[2]]) else None
except KeyError:
    gender_row = None

# Functions to convert variables
def extract_value(value):
    return value.split(': ')[1]

def convert_trait(value):
    converted_val = extract_value(value).lower()
    if "control" in converted_val:
        return 0
    elif "transgenic" in converted_val:
        return 1
    return None

def convert_age(value):
    try:
        return float(extract_value(value))
    except ValueError:
        return None

def convert_gender(value):
    converted_val = extract_value(value).lower()
    if "female" in converted_val:
        return 0
    elif "male" in converted_val:
        return 1
    return None

# Save Metadata
save_cohort_info('GSE147003', './preprocessed/Huntingtons_Disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Huntingtons_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Huntingtons_Disease/trait_data/GSE147003.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Huntingtons_Disease/cohort_info.json
{'GSM4412025': [0.0, 4.25, 0.0], 'GSM4412026': [1.0, 4.25, 0.0], 'GSM4412027': [0.0, 4.0, 0.0], 'GSM4412028': [1.0, 4.25, 1.0], 'GSM4412029': [0.0, 4.25, 0.0], 'GSM4412030': [1.0, 2.916666667, 0.0], 'GSM4412031': [0.0, 2.916666667, 0.0], 'GSM4412032': [1.0, 3.666666667, 0.0], 'GSM4412033': [0.0, 4.25, 1.0], 'GSM4412034': [1.0, 4.25, 0.0], 'GSM4412035': [0.0, 4.0, 0.0], 'GSM4412036': [1.0, 4.25, 1.0], 'GSM4412037': [0.0, 4.25, 1.0], 'GSM4412038': [1.0, 2.916666667, 0.0], 'GSM4412039': [0.0, 4.25, 1.0], 'GSM4412040': [1.0, 3.666666667, 0.0], 'GSM4412041': [0.0, 4.0, 0.0], 'GSM4412042': [1.0, 4.25, 0.0], 'GSM4412043': [0.0, 2.916666667, 1.0], 'GSM4412044': [1.0, 4.25, 1.0], 'GSM4412045': [0.0, 2.916666667, 1.0], 'GSM4412046': [1.0, 4.25, 0.0], 'GSM4412047': [0.0, 4.25, 0.0], 'GSM4412048': [1.0, 4.25, 0.0], 'GSM4412049': [0.0, 4.0, 0.0], 'GSM4412050': [1.0, 4.25, 0.0], 'GSM4412051': [0.0, 4.25, 0.0], 'GSM44